# Example 24: Multi-file Project Development

## Learning Objective
Learn to develop and manage multi-file Python projects with proper structure.

## Project Structure

```
task_manager/
├── __init__.py      # Package init
├── models.py        # Data models
├── repository.py    # Data access
├── service.py       # Business logic
└── tests/
    └── test_service.py
```

## Layer 1: Models

In [ ]:
# models.py
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum


class TaskStatus(Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"


@dataclass
class Task:
    """Represents a task."""
    title: str
    description: str = ""
    status: TaskStatus = TaskStatus.PENDING
    created_at: datetime = field(default_factory=datetime.now)
    id: int | None = None
    
    def complete(self):
        self.status = TaskStatus.COMPLETED

## Layer 2: Repository (Data Access)

In [ ]:
# repository.py
class TaskRepository:
    """In-memory storage for tasks."""
    
    def __init__(self):
        self._tasks = {}
        self._next_id = 1
    
    def add(self, task: Task) -> Task:
        """Add a task, assigning an ID."""
        task.id = self._next_id
        self._tasks[task.id] = task
        self._next_id += 1
        return task
    
    def get(self, task_id: int) -> Task | None:
        """Get task by ID."""
        return self._tasks.get(task_id)
    
    def get_all(self) -> list[Task]:
        """Get all tasks."""
        return list(self._tasks.values())
    
    def update(self, task: Task) -> bool:
        """Update an existing task."""
        if task.id in self._tasks:
            self._tasks[task.id] = task
            return True
        return False

## Layer 3: Service (Business Logic)

In [ ]:
# service.py
class TaskService:
    """Business logic for task management."""
    
    def __init__(self, repository: TaskRepository | None = None):
        self.repo = repository or TaskRepository()
    
    def create_task(self, title: str, description: str = "") -> Task:
        """Create a new task."""
        if not title.strip():
            raise ValueError("Title required")
        task = Task(title=title.strip(), description=description.strip())
        return self.repo.add(task)
    
    def complete_task(self, task_id: int) -> Task | None:
        """Mark a task as completed."""
        task = self.repo.get(task_id)
        if task:
            task.complete()
            self.repo.update(task)
        return task
    
    def get_pending_tasks(self) -> list[Task]:
        """Get all pending tasks."""
        return [t for t in self.repo.get_all() if t.status == TaskStatus.PENDING]

## Testing the Layers Together

In [ ]:
# Test the integrated system
service = TaskService()

# Create tasks
task1 = service.create_task("Write documentation")
task2 = service.create_task("Review code")
task3 = service.create_task("Deploy to production")

print("Created tasks:")
for task in service.repo.get_all():
    print(f"  [{task.id}] {task.title} - {task.status.value}")

# Complete one
service.complete_task(task1.id)

print(f"\nPending tasks: {len(service.get_pending_tasks())}")

## Benefits of Layered Architecture

1. **Separation of Concerns**: Each layer has one job
2. **Testability**: Test each layer independently
3. **Flexibility**: Swap implementations (e.g., database instead of memory)
4. **Maintainability**: Changes isolated to relevant layer

## The Prompt

```
Help me create a task management application with:
1. Task model with id, title, status, created_at
2. Repository for CRUD operations
3. Service with business logic
4. Unit tests

Organize as a proper Python package.
```

In [ ]:
# Practice: Add a "tags" feature
# 1. Add tags field to Task
# 2. Add filter_by_tag method to repository
# 3. Add get_tasks_by_tag to service